# Subplots

In [85]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Step 1：基本使用

这个例子中，我们会绘制三个变量的缺失率在一张图上，为了达成这个目的，需要利用```subplots```功能

In [86]:
data = pd.read_excel('data/data.xlsx')
data.sample(10, random_state=5)

,dataset,age,sex,smoking
33,十院,38.0,男,否
29,十院,12.0,男,否
44,儿中心,6.0,女,否
19,十院,25.0,女,是
40,儿中心,NaN,男,否
21,十院,27.0,女,是
6,十院,38.0,男,否
32,十院,45.0,女,是
2,十院,32.4,男,否
3,十院,33.5,女,NaN


## 创建figure

需要使用子图时，采用```make_subplots```功能

In [87]:
fig = make_subplots(
    rows=3, 
    cols=1
) # 3x1布局

var_names = data.columns[1:]
for dataset in data['dataset'].unique():
    sub_df = data.loc[data['dataset'] == dataset]

    for i, col in enumerate(var_names):
        null_rate = sub_df[col].isnull().mean() # 计算缺失率

        trace = go.Bar(
            x=['缺失', '非缺失'], 
            y=[null_rate, 1 - null_rate], 
            name=dataset
        )

        # 需要额外的指定row, col，明确trace添加的位置
        # 同样的位置可以多次添加trace
        fig.add_trace(
            trace,
            row=i + 1, #注意: col和row都是从1开始编号的
            col=1
        )

fig.update_layout(
    title='缺失率' # 这个是整张图的标题
)

fig


不过上面的图存在不少问题
* 颜色不统一
* 子图没有标题

下一个版本中，我们将会解决这些问题

# Step 2：

针对上面的问题，这个阶段我们作出如下的改进
* 子图的标题需要在创建subplots时就传入
* 每个数据集定义好固定的颜色

修改后的代码如下(需要调整的地方已经用注释标明。)

In [88]:
sub_titles = var_names
fig = make_subplots(
    rows=3, 
    cols=1,
    subplot_titles = var_names # 子图的标题需要在make_subplots时就提供。它们会以从左往右，从上往下的顺序的传递给子图
) # 3x1布局

# 数据集对应的颜色码
color_mapping = {
    '十院': '#ef563b',
    '儿中心': '#646cfb'
}

for dataset in data['dataset'].unique():
    sub_df = data.loc[data['dataset'] == dataset]

    for i, col in enumerate(var_names):
        null_rate = sub_df[col].isnull().mean()

        trace = go.Bar(
            x=['缺失', '非缺失'], 
            y=[null_rate, 1 - null_rate], 
            marker_color=color_mapping[dataset], # 这里基于数据集，指定预定义的颜色
            name="{}-{}".format(dataset, col)
        )

        fig.add_trace(
            trace,
            row=i + 1,
            col=1
        )

fig.update_layout(
    title='缺失率'
)

fig

有了不少改进，但是这个版本的lengend表现非常不自然。

实际上，我们只需要红，蓝两色区分两个数据集，并且控制相应的图形是否显示。

目前
* 每个trace都会生成的相应的图例，这非常冗余
* 我们无法通过点击一个图例，集中控制来自十院，或者儿中心的数据是否显示

下一个阶段的代码会解决这个问题

# Step 3

首先，我们实现集中控制数据集的信息是否显示

之前版本的代码中，几个红色的图例控件是相互独立的，点击一个，只会影响一个子图。

通过```lengendgroup```设置，我们可以让相关的lengend控件归属于同一个组。同一组的legend控件，点击任何一个控件，都会影响整个组。

调整后的代码如下

In [89]:
sub_titles = var_names
fig = make_subplots(
    rows=3, 
    cols=1,
    subplot_titles = var_names
)

color_mapping = {
    '十院': '#ef563b',
    '儿中心': '#646cfb'
}

for dataset in data['dataset'].unique():
    sub_df = data.loc[data['dataset'] == dataset]

    for i, col in enumerate(var_names):
        null_rate = sub_df[col].isnull().mean()

        trace = go.Bar(
            x=['缺失', '非缺失'], 
            y=[null_rate, 1 - null_rate], 
            marker_color=color_mapping[dataset],
            name="{}-{}".format(dataset, col),
            legendgroup = dataset # 我们希望来自同一个数据集的legend归属于同一组，天然的，可以用dataset的数值作为legendgroup的值
        )

        fig.add_trace(
            trace,
            row=i + 1,
            col=1
        )

fig.update_layout(
    title='缺失率'
)

fig

比较前面的版本，3个红色的lengend控件和3个蓝色的legend控件现在出现了间隔，这是分组成功的迹象。如果现在我们尝试点击legend控件，会发现会影响不在影响单个的trace，而是影响整个legend组。

但是目前，legend还是显得很冗余，对于每个数据集，其实我们只需要保留一个lengend控件。对此，我们可以用```.showlengend```属性来隐藏不必要的lengend控件

In [90]:
sub_titles = var_names
fig = make_subplots(
    rows=3, 
    cols=1,
    subplot_titles = var_names
)

color_mapping = {
    '十院': '#ef563b',
    '儿中心': '#646cfb'
}

for dataset in data['dataset'].unique():
    sub_df = data.loc[data['dataset'] == dataset]

    for i, col in enumerate(var_names):
        null_rate = sub_df[col].isnull().mean()

        trace = go.Bar(
            x=['缺失', '非缺失'], 
            y=[null_rate, 1 - null_rate], 
            marker_color=color_mapping[dataset],
            name="{}-{}".format(dataset, col),
            legendgroup = dataset
        )
        
        if i > 0: # 除非是第一个变量，否则隐藏lengend控件
            trace.showlegend=False

        fig.add_trace(
            trace,
            row=i + 1,
            col=1
        )

fig.update_layout(
    title='缺失率'
)

fig

大功告成！

# Step 4

为了更好的描述一个变换，除了缺失率，我们一般也需要观察变量取值的分布。因此，我们现在额外生成一列图表，描述变量取值的分布。

In [92]:
# 子图的标题除了区分变量，也需要区分可视化的内容是缺失率，还是数值分布？
from itertools import product 
sub_titles = [
    "{}-{}".format(var_name, item) 
    for var_name,item 
    in product(var_names, ['缺失率','数值分布'])
]

fig = make_subplots(
    rows=3, 
    cols=2, # 由于需要额外描述变量分布，我们生成3x2的布局
    subplot_titles = sub_titles
)


color_mapping = {
    '十院': '#ef563b',
    '儿中心': '#646cfb'
}

type_mapping = {
    'age':'number',
    'sex':'categorical',
    'smoking':'categorical'
}


for dataset in data['dataset'].unique():
    sub_df = data.loc[data['dataset'] == dataset]

    for i, col in enumerate(var_names):
        null_rate = sub_df[col].isnull().mean() # 计算缺失率

        trace = go.Bar(
            x=['缺失', '非缺失'], 
            y=[null_rate, 1 - null_rate], 
            marker_color=color_mapping[dataset],
            legendgroup=dataset,
            name=dataset
        )
        
        if i>0:
            trace.showlegend=False
            
        fig.add_trace(
            trace,
            row=i + 1,
            col=1
        )
        
        if type_mapping[col] =='number':
            trace = go.Histogram(
                x = sub_df[col],
                marker_color=color_mapping[dataset],              
                name=dataset,
                legendgroup=dataset,
                bingroup=col # 让同一个变量，来自两个数据集的histgram归属于同一个bingroup,确保划分方式一致
            )
            trace.showlegend=False
            
            fig.add_trace(
                trace,
                row=i + 1,
                col=2
            )            
        else:
            dist = sub_df[col].dropna().value_counts()
            trace = go.Bar(
                x = dist.index,
                y = dist,
                marker_color=color_mapping[dataset],            
                name=dataset,
                legendgroup=dataset
            )
            trace.showlegend=False
            
            fig.add_trace(
                trace,
                row=i + 1,
                col=2
            )

fig.update_layout(
    title='变量探索性分析'
)

# Step 5

目前的结果基本满足了功能性需求。然后如果是用于汇报，可视化结果的美观程度也是必须要考虑的因素。

这个阶段我们会做出以下改进
* 缺失率部分没必要占用一半的宽度，将缺失率，数值分布占用的宽度比例调整成2:8
* 整个图的标题加大字号
* 采用微软雅黑字体
* 缺失率采用x%的格式显示数值

In [149]:
# 子图的标题除了区分变量，也需要区分可视化的内容是缺失率，还是数值分布？
from itertools import product 
sub_titles = [
    "{}-{}".format(var_name, item) 
    for var_name,item 
    in product(var_names, ['缺失率','数值分布'])
]

fig = make_subplots(
    rows=3, 
    cols=2, 
    subplot_titles = sub_titles,
    column_widths=[0.3,0.7], # 调整宽度分配比例
    horizontal_spacing=0.05,
    vertical_spacing=0.1
)


color_mapping = {
    '十院': '#ef563b',
    '儿中心': '#646cfb'
}

type_mapping = {
    'age':'number',
    'sex':'categorical',
    'smoking':'categorical'
}


for dataset in data['dataset'].unique():
    sub_df = data.loc[data['dataset'] == dataset]

    for i, col in enumerate(var_names):
        null_rate = sub_df[col].isnull().mean() # 计算缺失率

        trace = go.Bar(
            x=['缺失', '非缺失'], 
            y=[null_rate, 1 - null_rate], 
            marker_color=color_mapping[dataset],
            legendgroup=dataset,
            name=dataset,
            width=0.3
        )
        
        if i>0:
            trace.showlegend=False
            
        fig.add_trace(
            trace,
            row=i + 1,
            col=1
        )
        
        if type_mapping[col] =='number':
            trace = go.Histogram(
                x = sub_df[col],
                marker_color=color_mapping[dataset],              
                name=dataset,
                legendgroup=dataset,
                bingroup=col, # 让同一个变量，来自两个数据集的histgram归属于同一个bingroup,确保划分方式一致
            )
            trace.showlegend=False
            
            fig.add_trace(
                trace,
                row=i + 1,
                col=2
            )
        else:
            dist = sub_df[col].dropna().value_counts()
            trace = go.Bar(
                x = dist.index,
                y = dist,
                marker_color=color_mapping[dataset],            
                name=dataset,
                legendgroup=dataset,
                width=0.1
            )
            trace.showlegend=False
            
            fig.add_trace(
                trace,
                row=i + 1,
                col=2
            )

fig.update_layout(
    template='plotly',
    height= 250*3,
    font={
        'family': 'Microsoft YaHei'
    },
    title={
        'text':'变量探索性分析',
        'font':{
            'size':25,
        },
        'x':0.5,
    }
)

for i in range(0, 6, 2):
    fig.update_layout(
        {
            'yaxis{}'.format(i+1):{
                'range':[0, 1],
                'tickformat':".1%",
                'tick0':0.0,
                'dtick':0.25
            }            
        }

         
    )    
    
fig